In [1]:
import os

os.environ['HTTP_PROXY'] = 'http://fp.cs.ovgu.de:3210/'
os.environ['HTTPS_PROXY'] = 'http://fp.cs.ovgu.de:3210/'

In [3]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from captum.attr import LayerGradCam, ScoreCAM, GradCAM, LayerCAM

ImportError: cannot import name 'ScoreCAM' from 'captum.attr' (/scratch/shkulkar/python/lib/python3.8/site-packages/captum/attr/__init__.py)

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "2,3"
ngpu = torch.cuda.device_count()
for gpu_id in range(ngpu):
    gpu = torch.cuda.get_device_name(gpu_id)
    print(f"GPU:{gpu_id} {gpu}")

In [ ]:
# Load a pre-trained model (for example, a ResNet model)
model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
model.eval()

In [ ]:
# Load an image
image_path = '/project/validating_attribution_techniques/commons/examples/input.png'
img = Image.open(image_path)

In [ ]:


# Preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

input_img = preprocess(img).unsqueeze(0)  # Add batch dimension



In [ ]:
# Define the attribution methods
layer_grad_cam = LayerGradCam(model, model.layer4[-1])
score_cam = ScoreCAM(model)
grad_cam = GradCAM(model, model.layer4[-1])
layer_cam = LayerCAM(model, model.layer4[-1])



In [ ]:
# Generate attribution maps
attribution_layer_grad_cam = layer_grad_cam.attribute(input_img)
attribution_score_cam = score_cam.attribute(input_img)
attribution_grad_cam = grad_cam.attribute(input_img)
attribution_layer_cam = layer_cam.attribute(input_img)



In [ ]:
# Plot the original image and attribution maps
fig, axes = plt.subplots(1, 5, figsize=(15, 5))

# Original Image
axes[0].imshow(np.transpose(input_img.squeeze().cpu().numpy(), (1, 2, 0)))
axes[0].set_title('Original Image')

# GradCAM
axes[1].imshow(attribution_grad_cam[0].cpu().numpy(), cmap='jet', alpha=0.5)
axes[1].imshow(np.transpose(input_img.squeeze().cpu().numpy(), (1, 2, 0)), alpha=0.4)
axes[1].set_title('GradCAM')

# ScoreCAM
axes[2].imshow(attribution_score_cam[0].cpu().numpy(), cmap='jet', alpha=0.5)
axes[2].imshow(np.transpose(input_img.squeeze().cpu().numpy(), (1, 2, 0)), alpha=0.4)
axes[2].set_title('ScoreCAM')

# Layer GradCAM
axes[3].imshow(attribution_layer_grad_cam[0].cpu().numpy(), cmap='jet', alpha=0.5)
axes[3].imshow(np.transpose(input_img.squeeze().cpu().numpy(), (1, 2, 0)), alpha=0.4)
axes[3].set_title('Layer GradCAM')

# LayerCAM
axes[4].imshow(attribution_layer_cam[0].cpu().numpy(), cmap='jet', alpha=0.5)
axes[4].imshow(np.transpose(input_img.squeeze().cpu().numpy(), (1, 2, 0)), alpha=0.4)
axes[4].set_title('LayerCAM')

plt.show()
